In [1]:
# content of test_sysexit.py
import pytest

def f():
    raise SystemExit(1)

def test_mytest():
    with pytest.raises(SystemExit):
        f()


#  db connection test#

# A list for available coins and bank notes
#  xxxx  # @pytest.fixture
def coins_notes():
    return [
        ['0.20', 0],   # 0.20
        ['0.50', 0],   # 0.50
        ['1', 0],      # 1
        ['2', 0],      # 2
        ['5', 0],      # 5
        ['10', 0],     # 10
        ['20', 0]      # 20
    ]


@pytest.fixture(scope='module')  # default scope=function , module, session
def db():
    print('------setup------')
    db = MySQLdb_connection()
    sql = "CREATE DATABASE IF NOT EXISTS %s" % 'mydb'
    db.query_db(sql)
    # return db
    yield db
    print('------teardown------')
    # db.close()


def test_load_mysql(db):
    sql1 = "SELECT 1 FROM mydb.coinsnotes LIMIT 1;"
    result1 = db.query_db(sql1, db_use='mydb')

    if not result1:
        # Create coinsnotes table
        list_data = coins_notes()

        # columns for coinsnotes table
        fields = ("id INT AUTO_INCREMENT PRIMARY KEY,"
                  "coin VARCHAR(5),"
                  "number INT(6)")

        sql = "CREATE TABLE IF NOT EXISTS %s (%s)" % ('mydb.coinsnotes', fields)
        r2 = db.query_db(sql, db_use='mydb')

        for l in list_data:

            sql = "INSERT INTO mydb.coinsnotes (coin, number) VALUES ('%s', %s);" % \
                (l[0], l[1])

            r2 = db.query_db(sql, db_use='mydb')

    sql = "select * from mydb.coinsnotes where coin = 0.20;"
    r2 = db.query_db(sql, db_use='mydb')
    # print(r2)
    expectd_op = (1, '0.20', 0)
    assert expectd_op in r2        
        
        

# pytest.mark.parametrize decorator
# enables parametrization of arguments for a test function.

# content of test_expectation.py


@pytest.mark.parametrize("test_input,expected", [
    ("3+5", 8),
    ("2+4", 6),
    ("6*9", 54),
])
def test_eval(test_input, expected):
    assert eval(test_input) == expected


# This will run the test with the arguments set to x=0/y=2, x=0/y=3, x=1/y=2 and x=1/y=3.
@pytest.mark.parametrize("x", [0, 1])
@pytest.mark.parametrize("y", [2, 3])
def test_foo(x, y):
    pass

# The ‘tmpdir’ fixture
# @pytest.fixture
# content of test_tmpdir.py


def test_create_file(tmpdir):
    print(tmpdir)
    p = tmpdir.mkdir("sub").join("hello.txt")
    p.write("content")
    assert p.read() == "content"
    assert len(tmpdir.listdir()) == 1
    # assert 0


In [ ]:
$ python -m basket -h

## Namespace(items=['apples', 'milk', 'soup'], products='products.json', 
## promotions='promotions.json', verbose=False)

## Examples
```
$ python -m basket apples milk
Subtotal: £2.30
Apples 10% off: -10p
Total: £2.20
```
```
$ python -m basket milk soup eggs --verbose
2016-12-17 14:03:02 INFO: Item 'eggs' not in stock
Subtotal: £1.95
(No offers available)
Total: £1.95
```
```
$ python -m basket milk soup soup bread apples
Subtotal: £4.40
2 tins soup get you a half price loaf: -40p
Apples 10% off: -10p
Total: £3.90
```

## Tests
This application benefits from `pytest` tests. To run the tests:
```bash
$ pytest basket   
or $ pytest basket -v(verbose) -s(show print)
or $ pytest ./tests/test_promotion.py -v
``` 

In [ ]:
## fix notes

// test_product.py
def test_prod(name, price, unit):
    prod = product.Product(name, price, unit)
    # assert prod.name == 'appels'
    assert prod.name == 'apples' #bugfix
    
// test_promotion.py
# @pytest.mark.parametrize('qualifying_qty', [1, 1.0, '1'])
@pytest.mark.parametrize('qualifying_qty', [1, 1.0, 1]) #bugfix

::test_bad_offer FAILED     <---  no parameter given
        
// test_main.py
def test_log_error(capsys):    
    # main.log('foo', level=main.ERROR)
    main.logger.log('foo', level=main.logger.error) #bugfix

def test_load_offers_bad_item...
    # assert '(Unacceptable value for qualifying_qty)' in stdout
    assert "('Unacceptable value for qualifying_qty')" in stdout #bugfix
    
def test_main(capsys):
    # expectd_op = 'Subtotal: £1.00\nApples 10% off: -10p\nTotal: £0.90'
    expectd_op = 'Subtotal: £1.00\nApples 10% off: -12p\nTotal: £0.88'
    
    


In [ ]:
#1# Implements the ability to distinguish between active and inactive products.

#- basket.py
    def isactive(self, item):
        """Check if an item is active.

        :param str item: The name of an item to be checked if active.
        :return: True if the item is active and False otherwise.
        """
        # print([p.active for p in self.items if p.name == item.lower()])
        try:
            prod_status = [p.active for p in self.items if p.name == item.lower()]
        except KeyError:
            return False
        return prod_status[0]
    
#- product.py
def __init__(self, name, price, unit, active):
    self.active = active
    
#- __main__.py
#     Make a basket and fill
#     shopping_basket = basket.Basket(products, promotions)
#     for item in args.items:
#         if not shopping_basket.add(item):
#             logger.log(f'Item \'{item}\' not in stock')
        else:
            r = shopping_basket.isactive(item)
            logger.log(f"Item {item} active status {r}")
            
#- products.json            
   , "active": true         
    
    
    
    
#2# Implement an option to output a 'till-receipt'.

#- __main__.py
    ...
    parser.add_argument(
        '-r',
        '--receipt',
        help='till-receipt output',
        default=False,
        action='store_true',
        dest='receipt',
    )
    
    
def till_receipt(sbasket):
    headers = ['Name', 'Price', 'Reduced Price']
    print('\n\t << Till Receipt >>')
    print("......................................")
    print('{}{}{}'.format(headers[0].ljust(10), headers[1].ljust(10), headers[2].ljust(10)))

    for prod in sbasket.items:
        print('{}{}{}'.format(prod.name.ljust(10), str(prod.price).ljust(10),
                              str(prod.discounted_price).ljust(10)))

    print(f'\nSubtotal: £{sbasket.subtotal/100:.2f}')
    print(f'Total: £{sbasket.total/100:.2f}')
    print("......................................")    

# def main(argv=None):
        
    
    # Till-Receipt required?
    if args.receipt:
        till_receipt(shopping_basket)

# if __name__ == '__main__':    
    
    
    
    
#3# Implement the notion of a 'time limited' promotion.
There is no right format;
The best format is arguably a date represented in ISO 8601 format 

JSON only knows these types:   string, number, object, array, true, false, null

#- promotions.json
"time_limited":"2020-02-21"
    
#- promotion.py
def __init__(self, promo_def):
    ...
    self.time_limited = promo_def['time_limited']
    
#- basket.py
import datetime
...
def calculate_discounts(self):
    ...
    # Check if the promotion is out of date
            if promotion.time_limited:
                datelimit = datetime.datetime.strptime(promotion.time_limited, "%Y-%m-%d").date()
                if datetime.date.today() > datelimit:
                    continue

#             # And how many times can we apply the discount?
#             num_discounts = qty_qualifying // promotion.qualifying_qty

#             if not num_discounts:
#                 continue
    

    
    
#4# implement a solution that uses an external SQL or NoSQL database.
To install MySQLdb on Windows go to this link – 
https://www.lfd.uci.edu/~gohlke/pythonlibs/#mysqlclient
pip install [.whl filename]
    -----------------------------
#- db_mysql_v2.py      new
import MySQLdb
# import logging
import os


class MySQLdb_connection(object):
    """mysql db connection"""

    def __init__(self, connection_string=os.environ["CONN"], db=''):
        # connect(host, user, passwd, db)

        connection_string = connection_string + ',' + db

        vars = connection_string.split(',')
        self.host = vars[0]
        self.user = vars[1]
        self.pw = vars[2]
        self.char = "utf8mb4"
        # print(connection_string)
        self.connector = None

        try:
            self.connector = MySQLdb.connect(host=self.host,
                                             user=self.user,
                                             passwd=self.pw,
                                             charset=self.char,)
        except Exception as e:
            print(e)

    def __del__(self):
        self.connector.close()
        # print('__del__')

    """   fetchall(),
      the return value is a sequence of "tuples" that contain
      the "row values".
    """

    def query_db(self, sql_cmd, db_use=''):
        r = None
        try:
            cur = self.connector.cursor()
            r = cur.execute(sql_cmd)
            # print(sql_cmd)
            if r != 0:
                r = cur.fetchall()

            self.connector.commit()
        except Exception as e:
            print(e)
            self.connector.rollback()
        finally:
            cur.close()
            # self.connector.close()
            if r:
                return r


# executes when your script is called from the command-line
if __name__ == "__main__":
    print('How to use?\n'
          'MySQLdb_connection.query_db("your-sql-cmd")')

    
------------------------------

#- __main__.py
from basket import db_mysql_v2 as db
...
mydb = db.MySQLdb_connection()
def load_mysql_data(args):
    """
    Setup here,    create database & create table
    """
    # db.MySQLdb_connection.create_db(db_name = 'mydb')
    sql = "CREATE DATABASE IF NOT EXISTS %s" % 'mydb'
    mydb.query_db(sql)


    sql1 ="SELECT 1 FROM mydb.products LIMIT 1;"
    result1 = mydb.query_db(sql1, db_use = 'mydb')
    sql2 ="SELECT 1 FROM mydb.promotions LIMIT 1;"
    result2 = mydb.query_db(sql2, db_use = 'mydb')

    if(result1 == False):
        # Create product table
        prods = load_json(args.products)
        # prods_obj = [prod for prod in prods]
        # keys = [key for key in prods[0]]

        # columns for products table
        fields = ("id INT AUTO_INCREMENT PRIMARY KEY,"
                  "name VARCHAR(255),"
                  "price INT(6),"
                  "unit VARCHAR(255),"
                  "active boolean DEFAULT false")

        sql = "CREATE TABLE IF NOT EXISTS %s (%s)" % ('mydb.products', fields)
        r2 = mydb.query_db(sql, db_use = 'mydb')

        for prod in prods:
            # split dictionary into keys and values
            k, values = zip(*prod.items())

            sql = "INSERT INTO mydb.products (name, price, unit, active) VALUES ('%s',%s,'%s',%s);" % \
            (values[0],values[1],values[2],int(values[3]))

            r2 = mydb.query_db(sql, db_use = 'mydb')

    elif(result2 == False):
        # Create promotions table
        promos = load_json(args.promotions)

        pfields = ('id INT PRIMARY KEY,'
                   'title VARCHAR(255),'
                   'qualifying_product VARCHAR(255),'
                   'qualifying_qty INT(6),'
                   'discounted_product VARCHAR(255),'
                   'discount_percent INT(6),'
                   'time_limited VARCHAR(12)')

        sql = "CREATE TABLE IF NOT EXISTS %s (%s)" % ('mydb.promotions', pfields)
        r2 = mydb.query_db(sql, db_use = 'mydb')

        for promo in promos:
            # split dictionary into keys and values
            k, values = zip(*promo.items())
            # cols = ','.join(k)
            # print(cols)

            sql = "INSERT INTO mydb.promotions \
            (id,title,qualifying_product,qualifying_qty,discounted_product,\
            discount_percent,time_limited) \
            VALUES (%s,'%s','%s',%s,'%s',%s,'%s');" % \
            (values[0],values[1],values[2],values[3], values[4], values[5], values[6])

            r2 = mydb.query_db(sql, db_use = 'mydb')


#5# Update the application to use the Logbook logging library.

#- __main__.py
from logbook import Logger, StreamHandler

StreamHandler(sys.stdout).push_application()
logger = Logger('Basket')


# class SimpleLogger:
#     enabled = False
#     info = 'INFO'
#     error = 'ERROR'
#
#     def log(self, message, level=info):
#         """Simple logger.
#
#         :param str message: Log message
#         :param str level: Log level name
#         """
#         if self.enabled:
#             now = time.strftime('%Y-%m-%d %H:%M:%S')
#             print('{} {}: {}'.format(now, level, message))
#
#
# logger = SimpleLogger()

-- just after main() --

    # logger.enabled = args.verbose
    if not args.verbose:
        logger.disable()

--- change all the 'logger.log '  to     logger.error, logger.warn, etc ---  

#     Make a basket and fill
#     shopping_basket = basket.Basket(products, promotions)
#     for item in args.items:
#         if not shopping_basket.add(item):
            logger.warn(f'Item \'{item}\' not in stock')
#         else:
#             r = shopping_basket.isactive(item)
            logger.info(f"Item {item} active status {r}")

    
    
#6# Design a REST API so the app can be provided as a service.

1, http://[hostname]/todo/api/v1.0/
    
2, Our tasks resource will use HTTP methods as follows:

HTTP Method                                          URI                                                      Action
GET           http://[hostname]/todo/api/v1.0/products                         Retrieve list of products 
GET           http://[hostname]/todo/api/v1.0/products/[name]          Retrieve a product
GET           http://[hostname]/todo/api/v1.0/promotions                    Retrieve list of promotions  
GET           http://[hostname]/todo/api/v1.0/promotions/[id]           Retrieve a promotion
POST         http://[hostname]/todo/api/v1.0/products                         Create a new products
PUT           http://[hostname]/todo/api/v1.0/products/[name]          Update an existing products
DELETE   http://[hostname]/todo/api/v1.0/products/[name]          Delete a products    
    
    

In [ ]:
#!env/Scripts/python

# https://blog.miguelgrinberg.com/post/designing-a-restful-api-with-python-and-flask

from flask import Flask, jsonify
from flask import abort
from flask import make_response
from flask import request
from flask_httpauth import HTTPBasicAuth
auth = HTTPBasicAuth()

import json
import os

app = Flask(__name__)

@app.errorhandler(404)
def not_found(error):
    return make_response(jsonify({'error': 'Not found'}), 404)

# @app.route('/')
# def index():
#     return "Hello, World!"


# tasks = [
#     {
#         'id': 1,
#         'title': u'Buy groceries',
#         'description': u'Milk, Cheese, Pizza, Fruit, Tylenol',
#         'done': False
#     },
#     {
#         'id': 2,
#         'title': u'Learn Python',
#         'description': u'Need to find a good Python tutorial on the web',
#         'done': False
#     }
# ]

# @app.route('/todo/api/v1.0/products', methods=['GET'])
# def get_tasks():
#     return jsonify({'tasks': tasks})

"""Load json data.

products,
promotions,
"""
products = None
promotions = None

# script_path = os.path.abspath(__file__)  #path to directory/app.py
# script_dir = os.path.split(script_path)[0] #path to dir/
# abs_products_path = os.path.join(script_dir, 'products.json')
# abs_promotions_path = os.path.join(script_dir,'promotions.json')

try:
    # with open(abs_products_path) as f1, \
    #     open(abs_promotions_path) as f2:
    with open('../products.json') as f1, \
        open('../promotions.json') as f2:
        products = json.load(f1)
        promotions = json.load(f2)
except EnvironmentError:
    # logger.log(f'No such file or directory: {json_file_path}',
    #            SimpleLogger.error)
    print('No such file or directory')
except json.JSONDecodeError as e:
    # logger.log(f'Failed to parse data file {json_file_path}: {e}',
    #            SimpleLogger.error)
    print('Failed to parse data file')

def write_jsonfile(file_path):
    try:
        with open(file_path, 'w') as jsonfile:
            json.dump(products, jsonfile)
    except EnvironmentError:
        print(f'No such file or directory, {file_path}')
    except json.JSONDecodeError as e:
        print(f'Failed to parse data file, {file_path}')



# curl -i http://127.0.0.1:5000/
@app.route('/todo/api/v1.0/products', methods=['GET'])
def get_products():
    return jsonify({'products': products})

@app.route('/todo/api/v1.0/promotions', methods=['GET'])
def get_promotions():
    return jsonify({'promotions': promotions})

@app.route('/todo/api/v1.0/products/<string:product_name>', methods=['GET'])
def get_product(product_name):
    product = [prod for prod in products if prod['name'] == product_name]
    if len(product) == 0:
        abort(404)
    return jsonify({'product': product[0]})

@app.route('/todo/api/v1.0/promotions/<int:promotion_id>', methods=['GET'])
def get_promotion(promotion_id):
    promotion = [prom for prom in promotions if prom['id'] == promotion_id]
    if len(promotion) == 0:
        abort(404)
    return jsonify({'promotion': promotion[0]})


# curl -i -H "Content-Type: application/json" -X POST
# -d '{"name": "Coffee", "price": 130, "unit": "Cup", "active": true}'
# http://localhost:5000/todo/api/v1.0/products
@app.route('/todo/api/v1.0/products', methods=['POST'])
def create_product():
    if not request.json or not 'name' in request.json:
        abort(400)
    else:
        pname = request.json['name']
        for p in products:
            if pname == p['name']:
                abort(400)

    product = {
        'name': request.json['name'],
        'price': request.json['price'],
        'unit': request.json['unit'],
        'active': request.json['active']
    }
    products.append(product)

    write_jsonfile('../products.json')

    return jsonify({'product': product}), 201

@app.route('/todo/api/v1.0/promotions', methods=['POST'])
def create_promotion():
    pass



# curl -i -H "Content-Type: application/json" -X PUT
# -d '{"name": "Coffee", "price": 130, "unit": "Cup", "active": true}'
# http://localhost:5000/todo/api/v1.0/products/Coffee
@app.route('/todo/api/v1.0/products/<string:product_name>', methods=['PUT'])
def update_product(product_name):
    product = [prod for prod in products if prod['name'] == product_name]
    if len(product) == 0:
        abort(404)
    if not request.json:
        abort(400)
    # if 'name' in request.json and type(request.json['name']) != unicode:
    #     abort(400)
    if 'active' in request.json and type(request.json['active']) is not bool:
        abort(400)
    product[0]['name'] = request.json.get('name', product[0]['name'])
    product[0]['price'] = request.json.get('price', product[0]['price'])
    product[0]['unit'] = request.json.get('unit', product[0]['unit'])
    product[0]['active'] = request.json.get('active', product[0]['active'])

    write_jsonfile('../products.json')

    return jsonify({'product': product[0]})

# curl -u miguel:python -i -H "Content-Type: application/json" -X DELETE
# -d '{"name": "Coffee"}'
# http://localhost:5000/todo/api/v1.0/products/Coffee
@app.route('/todo/api/v1.0/products/<string:product_name>', methods=['DELETE'])
@auth.login_required
def delete_product(product_name):
    product = [prod for prod in products if prod['name'] == product_name]
    if len(product) == 0:
        abort(404)
    products.remove(product[0])

    write_jsonfile('../products.json')

    return jsonify({'result': True})


"""
The easiest way to secure our web service is to require clients to provide
a username and a password. In a regular web application you would have
a login form that posts the credentials, and at that point the server would
create a session for the logged in user to continue working, with the
session id stored in a cookie in the client browser.
Unfortunately doing that here would violate the stateless requirement of REST,
so instead we have to ask clients to send their authentication information
with every request they send to us.
"""
@auth.get_password
def get_password(username):
    if username == 'miguel':
        return 'python'  # userinfo should be retrieved from DB or safer way.
    return None

@auth.error_handler
def unauthorized():
    # return make_response(jsonify({'error': 'Unauthorized access'}), 401)
    return make_response(jsonify({'error': 'Unauthorized access'}), 403)


if __name__ == '__main__':
    app.run(debug=True)
    

In [17]:
import datetime
a = '2010-01-31'

datee = datetime.datetime.strptime(a, "%Y-%m-%d").date()


datee.month

datee.year

datee.day

if datetime.date.today() > datee:
    print(datee)


2010-01-31


datetime.date(2020, 1, 31)